# Коэффициенты корреляции

In [20]:
import pandas as pd
import numpy as np
from scipy.stats import *

## Корреляция Пирсона и Спирмена

Для 61 большого города в Англии и Уэльсе известны средняя годовая смертность на 100000 населения (по данным 1958–1964) и концентрация кальция в питьевой воде (в частях на маллион). Чем выше концентрация кальция, тем жёстче вода. Города дополнительно поделены на северные и южные.<br>
Есть ли связь между жёсткостью воды и средней годовой смертностью? 

In [3]:
data=pd.read_csv('data/water.txt', header=0, sep='\t')
data.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


Посчитаем коэффициенты корреляции Пирсона и Спирмена.

In [9]:
data.corr()

,mortality,hardness
mortality,1.000000,-0.654849
hardness,-0.654849,1.000000


In [7]:
data.corr(method='spearman')

,mortality,hardness
mortality,1.000000,-0.631665
hardness,-0.631665,1.000000


Проверим, сохраняется ли связь между признаками, если разбить выборку на северные и южные города.

In [17]:
data[data['location']=='South'].corr()

,mortality,hardness
mortality,1.000000,-0.602153
hardness,-0.602153,1.000000


In [18]:
data[data['location']=='North'].corr()

,mortality,hardness
mortality,1.000000,-0.368598
hardness,-0.368598,1.000000


Как видим, в подвыборке северных городов признаки уже оказываются не настолько сильно коррелированными.

## Корреляция Мэтьюса

Среди респондентов General Social Survey 2014 года хотя бы раз в месяц проводят вечер в баре 203 женщины и 239 мужчин; реже, чем раз в месяц, это делают 718 женщин и 515 мужчин.<br>
Рассчитаем значение коэффициента корреляции Мэтьюса между полом и частотой похода в бары.

In [26]:
contingency_table=np.array([[203,239],[718,515]])
MCC=(203*515-718*239)/np.sqrt(921*442*754*1233)
print('Коэффициент корреляции Мэтьюса: %.4f'%MCC)

Коэффициент корреляции Мэтьюса: -0.1090


Проверим, значимо ли полученный коэффициент корреляции Мэтьюса отличается от нуля. 

In [28]:
hi2, p, dof, exp=chi2_contingency(contingency_table)
print('Достигаемый уровень значимости коэф. коррреляции Мэтьюса: %.8f'%p)

Достигаемый уровень значимости коэф. коррреляции Мэтьюса: 0.00001056


На уровне значимости 0.05 нулевая гипотеза, что коэффициент Мэтьюса получился незначимым, отвергается.

In [38]:
def proportion_diff_confint(table,alpha):
    n1=table[:,0].sum()
    n2=table[:,1].sum()
    p1=table[0,0]/n1 
    p2=table[0,1]/n2
    z=norm.ppf(1-alpha/2)
    upper=p2-p1 + z*np.sqrt(p1*(1-p1)/n1 + p2*(1-p2)/n2)
    lower=p2-p1 - z*np.sqrt(p1*(1-p1)/n1 + p2*(1-p2)/n2)
    return (lower,upper)

In [36]:
print('95%% доверительный интервал для разности долей мужчин и женщин, часто проводящих время в баре: [%.4f, %.4f].'\
      %proportion_diff_confint(contingency_table, 0.05))

95% доверительный интервал для разности долей мужчин и женщин, часто проводящих время в баре: [0.0539, 0.1392].


Проверим, отличаются ли доли любителей баров среди мужчин и женщин.<br>
$H_0$: их доли одинаковы.<br>
$H_1$: их доли отличаются.<br>
Для этого будем использовать z-критерий для двух несвязанных выборок.

In [43]:
def zstat_prop_diff_ind(table,alpha):
    n1=table[:,0].sum()
    n2=table[:,1].sum()
    p1=table[0,0]/n1 
    p2=table[0,1]/n2
    p=(p1*n1+p2*n2)/(n1+n2)
    z=(p1-p2)/np.sqrt(p*(1-p)*(1/n1+1/n2))
    return z

In [44]:
def pvalue(stat,h1):
    if h1=='two-sided':
        return 2*(1-norm.cdf(np.fabs(stat)))
    elif h1=='greater':
        return 1-norm.cdf(stat)
    elif h1=='less':
        return norm.cdf(stat)

In [46]:
print('Достигаемый уровень значимости: %.8f'%pvalue(zstat_prop_diff_ind(contingency_table,0.05), 'two-sided'))


Достигаемый уровень значимости: 0.00000815


p-value очень мал, что дает нам право отвергнуть нулевую гипотезу.